In [40]:
import sys
import os
import re
import json
from pathlib2 import Path
import os.path 

import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime 

import numpy as np 
import cv2 
import lmfit
from lmfit import Model, Parameters
from scipy.optimize import minimize

In [2]:
#  Import du csv generer de clean_csv.ipynb
df_main_board = pd.read_csv('../data/board_info_csv/processed/board_data_2F2.csv')
df_top = df_main_board[df_main_board['Layer'] == 'TopLayer'].copy() 

In [5]:
main_path = Path('../data/data_default_processed/')
prog = re.compile(r'2\.F\.2_G(\d+)\s*-\s*([\w\s]+Top[\w\s]*)_PP(\d+)\.jpg')

In [6]:
counter = -1
all_files = []
for file_path in main_path.glob('2.F.2*.jpg'):
    file_name = file_path.name
    # print(file_name)
    match = prog.match(file_name)
    
    if match: 
        board_id = int(match.group(1))
        orientation = match.group(2)
        photo_id = int(match.group(3))
        
        all_files.append({'id': board_id, 'path': str(file_path), 'orientation': orientation, 'photo_id': photo_id})

print(all_files)


[{'id': 156, 'path': '../data/data_default_processed/2.F.2_G156 - F2BFB6528B_Top_PP2.jpg', 'orientation': 'F2BFB6528B_Top', 'photo_id': 2}, {'id': 156, 'path': '../data/data_default_processed/2.F.2_G156 - F2BFB6528B_Top_PP1.jpg', 'orientation': 'F2BFB6528B_Top', 'photo_id': 1}, {'id': 306, 'path': '../data/data_default_processed/2.F.2_G306 - 3C890ADF5F_Top_PP1.jpg', 'orientation': '3C890ADF5F_Top', 'photo_id': 1}, {'id': 149, 'path': '../data/data_default_processed/2.F.2_G149 - 4A22055D30_Top_PP1.jpg', 'orientation': '4A22055D30_Top', 'photo_id': 1}, {'id': 177, 'path': '../data/data_default_processed/2.F.2_G177 - 8F13819335_Top_PP1.jpg', 'orientation': '8F13819335_Top', 'photo_id': 1}, {'id': 296, 'path': '../data/data_default_processed/2.F.2_G296 - 5EF56B3888_Top_PP1.jpg', 'orientation': '5EF56B3888_Top', 'photo_id': 1}, {'id': 240, 'path': '../data/data_default_processed/2.F.2_G240 - 0ACCC7D22F_Top_PP1.jpg', 'orientation': '0ACCC7D22F_Top', 'photo_id': 1}, {'id': 244, 'path': '../da

In [38]:
df_top[df_top['Designator']=='J2_1']

,Part Number,Comment,Description,Designator,Footprint,Layer,Ref-X(mm),Ref-Y(mm),Rotation,dimX,dimY,dimglueX,dimglueY,max_X,max_Y
1058,790-0014,61083-101400LF,CONN PLUG Mezzanine VERTICAL FEMALE 2X50 P0.8mm,J2_1,CON-PLUG-STRAIGHT-VER-61083-101400LF-AMP,TopLayer,89.55,65.875,0,46.0,4.95,39.200079,5.10101,46.0,5.10101


In [133]:
# Define the template matching method
def get_matching_method():
    methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR',
               'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
    return eval(methods[1])

# Load an image and a pattern for template matching
def load_images(image_path, pattern_path):
    image = cv2.imread(image_path, 1)
    pattern_check = cv2.imread(pattern_path, 1)
    pattern_check_gray = cv2.cvtColor(pattern_check, cv2.COLOR_BGR2GRAY)
    return image, pattern_check_gray

# Perform template matching and return the matched results
def perform_template_matching(image, pattern_check_gray):
    method = get_matching_method()

    gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
    res = cv2.matchTemplate(gray, pattern_check_gray, method)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    # Centering the matched result on component and scaling it to mm
    matched_result = [(max_loc[0] + (pattern_check_gray.shape[1]/2)), (max_loc[1] + (pattern_check_gray.shape[0]/2))]
    
    print(f"Matched Result: {matched_result}")    
    cv2.rectangle(image, max_loc, (max_loc[0] + pattern_check_gray.shape[1], max_loc[1] + pattern_check_gray.shape[0]), (0,0,0), 2, 8, 0 )
    cv2.imwrite('all_images_with_match/image_matched_{}.jpg'.format(board_id), image)

    return matched_result

# Fit the matched results to the csv data
def fit_results(matched_result_J2, matched_result_J704, matched_result_BT100, is_top, df_top):
    if is_top:
        # Coord of csv
        component_J2 = df_top[df_top["Designator"]=="J2_1"]
        x_J2 = component_J2['Ref-X(mm)'].values[0]
        y_J2 = component_J2['Ref-Y(mm)'].values[0] 
        
        component_J704 = df_top[df_top["Designator"]=="J704_1"]
        x_J704 = component_J704['Ref-X(mm)'].values[0]
        y_J704 = component_J704['Ref-Y(mm)'].values[0] 

        component_BT100 = df_top[df_top["Designator"]=="BT100_1"]
        x_BT100 = component_BT100['Ref-X(mm)'].values[0]
        y_BT100 = component_BT100['Ref-Y(mm)'].values[0] 
        
        csv_points = np.array([[x_J2, y_J2], [x_J704, y_J704], [x_BT100, y_BT100]], dtype=np.float32)

        # Coord of template
        template_points = np.array([matched_result_J2, matched_result_J704, matched_result_BT100], dtype=np.float32)

        # Calculate transformation matrix
        transformation_matrix, _ = cv2.estimateAffine2D(csv_points, template_points)
        
        print(f'Transformation Matrix: {transformation_matrix}')

    return transformation_matrix

# Process the matched results and create component rectangles
def process_matched_results(image, matched_results_J2,matched_results_J704, matched_results_BT100, df_top, is_top): 
    transformation_matrix = fit_results(matched_results_J2, matched_results_J704, matched_results_BT100, is_top, df_top) 
    components_to_process = ["J2_1", "U701_1", "U911U5_1"]
        

    # Extract rectangles around the components in components_to_process
    for component in components_to_process:
        # Get the component's X and Y coordinates from the CSV file
        component_data = df_top[df_top["Designator"]==component]
        x = component_data['Ref-X(mm)'].values[0]
        y = component_data['Ref-Y(mm)'].values[0]

        new_point = np.array([[x, y]], dtype=np.float32)
        new_camera_point = cv2.transform(new_point.reshape(1, -1, 2), transformation_matrix).squeeze()

        print(f"New Camera Point: {new_camera_point}")

        # Load the pattern image
        modified_comp_string = component.replace("_1", "")
        pattern_path = f'../data/template_images/{modified_comp_string}.jpg'
        pattern_check_component = cv2.imread(pattern_path, 1)  
        
        Width = int(pattern_check_component.shape[1]/2)  
        Height = int(pattern_check_component.shape[0]/2)

        # cv2.rectangle(image, max_loc, (max_loc[0] + pattern_check_gray.shape[1], max_loc[1] + pattern_check_gray.shape[0]), (0,0,0), 2, 8, 0 )
        cv2.rectangle(image, (int(new_camera_point[0]) - Width, int(new_camera_point[1]) - Height), (int(new_camera_point[0]) + Width, int(new_camera_point[1]) + Height),(255,0,0), 2, 8, 0)
        

    # Save the image with the rectangles
    cv2.imwrite(f'all_images_with_match/image_matched_{board_id}_6.jpg', image)


# Main function for processing images
def process_images(all_files, df_top):
    board_already_done = []
    status_bar = tqdm(total=len(all_files))

    is_top = True
    is_PP1 = True
    is_2F2 = True

    for a_pic in all_files[0:1]:
        status_bar.update()

        board_id = a_pic['id']
        orientation = a_pic['orientation']
        photo_PP_id = a_pic['photo_id']
        board_already_done.append(board_id)
  
        image, pattern_check_gray_J2 = load_images(a_pic['path'], '../data/template_images/J2.jpg') 
        image, pattern_check_gray_J704 = load_images(a_pic['path'], '../data/template_images/J704.jpg')
        image, pattern_check_gray_BT100 = load_images(a_pic['path'], '../data/template_images/BT100.jpg')
        matched_result_J2 = perform_template_matching(image, pattern_check_gray_J2)
        matched_result_J704 = perform_template_matching(image, pattern_check_gray_J704)
        matched_result_BT100 = perform_template_matching(image, pattern_check_gray_BT100)
        process_matched_results(image, matched_result_J2, matched_result_J704, matched_result_BT100, df_top, is_top)   
  
    status_bar.close()

In [134]:
process_images(all_files, df_top)

  0%|          | 0/28 [00:00<?, ?it/s]

Matched Result: [4926.5, 3689.5]
Matched Result: [1532.0, 2305.0]
Matched Result: [7255.5, 1835.5]
Transformation Matrix: [[ 4.73543745e+01 -2.54471522e+00  8.53548733e+02]
 [-1.38510802e-01 -4.88275242e+01  6.91841680e+03]]
New Camera Point: [4926.5    3689.5002]
New Camera Point: [1392.5042 5627.5977]
New Camera Point: [4045.2822 4584.5757]
